In [1]:
# import necessary packages
import sys
import os, time
import torch 
import numpy as np
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, utils
import random, matplotlib
import pandas as pd
#from torch.utils.tensorboard import SummaryWriter
#from torchvision.models.resnet import BasicBlock
#import torch.utils.model_zoo as model_zoo

import scipy.io
import h5py
import gc
from sklearn.model_selection import train_test_split
from PIL import Image
sys.path.append('./models/')
sys.path.append('./source/')

import utils as myUtils
from DataTypes import EEGDataset
from CNN_1 import Network

# Check does your computer support using GPU
print("GPU_available={}".format(torch.cuda.is_available()))

GPU_available=True


In [2]:
# This code is for training multiple models using Leave One Out (LOU)

# EEG_file_types = [2,1,0]
EEG_file_types = [10]

for EEG_file_type in EEG_file_types:

    trial = "trial21"
    folder = "./Results/"+trial+"/1FC_layer/"

    if(EEG_file_type == 0):
        mat = scipy.io.loadmat('EEG_samples_raw.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_raw/"
    elif(EEG_file_type == 3):
        sub_folder = "Kfold_noise_pruned/"
    elif(EEG_file_type == 2):
        mat = scipy.io.loadmat('EEG_samples_pruned.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_pruned/"
    else:
        mat = scipy.io.loadmat('EEG_samples_roi.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_roi/"

    folder = folder + sub_folder
    no_participants = 29
    train_data_set, train_label_set = [], []
    epoches = 550

    for partic_id in range(27,no_participants):
        fileName = "Subject_"+str(partic_id+1)+"_Results.txt"
        testFileName = "Test_Results.txt"
        path = folder+fileName
        test_path = folder+testFileName

        # Spilt the data into training and testing sets 
        if(EEG_file_type == 3):
            full_data, train_data, test_data = myUtils.generateTrainTestByFile('EEG_samples_noise_pruned.mat', partic_id, normalize = True, LOU = False)
        else:
            full_data, train_data, test_data = myUtils.generateTrainTest(EEG_samples, partic_id, normalize = False, LOU = True)

        # Save information concerning the data to file
        myUtils.writeStatsToFile(full_data, train_data, test_data, path)

        # Create a torch data loader for both the training and testing sets
        train_loader = myUtils.generateTorchLoaders(train_data[0], train_data[1], EEGDataset)
        test_loader = myUtils.generateTorchLoaders(test_data[0], test_data[1], EEGDataset)

        # Free up memory space
        del full_data
        del train_data
        del test_data
        gc.collect()


        # Create Instance of CNN model
        net = Network(num_classes=2)

        # Train the model
        print("Training on-going for participant "+str(partic_id+1))
        net = myUtils.train(net, train_loader, test_loader, epoches, path)

        state_dict = net.state_dict()
        torch.save(state_dict, 'trained_models/'+trial+'/'+sub_folder+'Arch'+str(partic_id+1)+'.pth')

        #Test the model
        acc_test = myUtils.eval(net, test_loader, test_path)

        # Free up memory space
        del train_loader
        del test_loader
        del net
        gc.collect()



Training on-going for participant 28
Cuda is Avaliable


c:\Users\Abdulsatar\Documents\UBICOMP\Thesis Work\ReverseNeuro\./models\CNN_1.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax8(x)


epoch: 0, lr: 0.010000, accuracy: 0.511362, loss: 0.688625, valid accuracy: 0.573333
epoch: 1, lr: 0.010000, accuracy: 0.532965, loss: 0.690991, valid accuracy: 0.844444
epoch: 2, lr: 0.010000, accuracy: 0.552168, loss: 0.683769, valid accuracy: 0.335556
epoch: 3, lr: 0.010000, accuracy: 0.554169, loss: 0.688158, valid accuracy: 0.204444
epoch: 4, lr: 0.010000, accuracy: 0.567531, loss: 0.673731, valid accuracy: 0.386667
epoch: 5, lr: 0.010000, accuracy: 0.575212, loss: 0.682372, valid accuracy: 0.195556
epoch: 6, lr: 0.010000, accuracy: 0.578893, loss: 0.669249, valid accuracy: 0.273333
epoch: 7, lr: 0.010000, accuracy: 0.589454, loss: 0.682779, valid accuracy: 0.626667
epoch: 8, lr: 0.010000, accuracy: 0.589774, loss: 0.653706, valid accuracy: 0.726667
epoch: 9, lr: 0.010000, accuracy: 0.603937, loss: 0.670313, valid accuracy: 0.380000
epoch: 10, lr: 0.010000, accuracy: 0.614098, loss: 0.674028, valid accuracy: 0.422222
epoch: 11, lr: 0.010000, accuracy: 0.617219, loss: 0.643296, val

In [3]:
# This code is fot training a single model

# EEG_file_types = [2,1,0]
EEG_file_types = [2]

for EEG_file_type in EEG_file_types:

    trial = "trial24"
    folder = "./Results/"+trial+"/1FC_layer/"

    if(EEG_file_type == 0):
        mat = scipy.io.loadmat('EEG_samples_raw.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_raw/"
    elif(EEG_file_type == 3):
        sub_folder = "Kfold_noise_pruned/"
    elif(EEG_file_type == 2):
        mat = scipy.io.loadmat('EEG_samples_pruned.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_pruned/"
    else:
        mat = scipy.io.loadmat('EEG_samples_roi.mat')
        EEG_samples = mat['EEG_samples']
        sub_folder = "Kfold_roi/"

    folder = folder + sub_folder
    no_participants = 29
    train_data_set, train_label_set = [], []
    epoches = 550

   
    fileName = "Results.txt"
    testFileName = "Test_Results.txt"
    path = folder+fileName
    test_path = folder+testFileName


    full_data, train_data, test_data = myUtils.generateTrainTest(EEG_samples, 1, normalize = True, LOU = False)

    # Save information concerning the data to file
    myUtils.writeStatsToFile(full_data, train_data, test_data, path)

    # Create a torch data loader for both the training and testing sets
    train_loader = myUtils.generateTorchLoaders(train_data[0], train_data[1], EEGDataset)
    test_loader = myUtils.generateTorchLoaders(test_data[0], test_data[1], EEGDataset)

    # Free up memory space
    del full_data
    del train_data
    del test_data
    gc.collect()


    # Create Instance of CNN model
    net = Network(num_classes=2)

    # Train the model
    print("Training on-going for participant ")
    net = myUtils.train(net, train_loader, test_loader, epoches, path)

    state_dict = net.state_dict()
    torch.save(state_dict, 'trained_models/'+trial+'/Arch.pth')

    #Test the model
    acc_test = myUtils.eval(net, test_loader, test_path)

    # Free up memory space
    del train_loader
    del test_loader
    del net
    gc.collect()



Minimum Value: -99.75249481201172
Maximum Value: 99.46538543701172
Training on-going for participant 
Cuda is Avaliable
epoch: 0, lr: 0.010000, accuracy: 0.493118, loss: 0.693261, valid accuracy: 0.507058
epoch: 1, lr: 0.010000, accuracy: 0.504070, loss: 0.693203, valid accuracy: 0.457331
epoch: 2, lr: 0.010000, accuracy: 0.503034, loss: 0.693218, valid accuracy: 0.456047
epoch: 3, lr: 0.010000, accuracy: 0.503034, loss: 0.693155, valid accuracy: 0.456689
epoch: 4, lr: 0.010000, accuracy: 0.501406, loss: 0.692890, valid accuracy: 0.454604
epoch: 5, lr: 0.010000, accuracy: 0.502590, loss: 0.693023, valid accuracy: 0.456368
epoch: 6, lr: 0.010000, accuracy: 0.502886, loss: 0.693206, valid accuracy: 0.456689
epoch: 7, lr: 0.010000, accuracy: 0.503182, loss: 0.693062, valid accuracy: 0.454443
epoch: 8, lr: 0.010000, accuracy: 0.501850, loss: 0.693267, valid accuracy: 0.455085
epoch: 9, lr: 0.010000, accuracy: 0.504218, loss: 0.692984, valid accuracy: 0.464228
epoch: 10, lr: 0.010000, accur